In [1]:
import requests
import pandas as pd

url = "https://www.nike.com/es/w?q=metcon&vst=metcon"

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Referer': 'https://www.google.com'
}
response = requests.get(url,headers=headers)
from bs4 import BeautifulSoup
soup=BeautifulSoup(response.content, "html.parser")

In [ ]:
from datetime import datetime
import pandas as pd
from bs4 import BeautifulSoup
import time
import re
"""
"""
def clean_price(price_str):
    """
    Limpia la cadena de precio, eliminando símbolos no numéricos.
    
    Args:
        price_str (str): Cadena con el precio a limpiar.
    
    Returns:
        float: Precio convertido a número.
    """
    # Eliminar símbolos de moneda, espacios especiales y reemplazar coma por punto
    clean_str = re.sub(r'[^\d,.]', '', price_str)
    
    # Reemplazar coma por punto si es un separador decimal
    clean_str = clean_str.replace(',', '.')
    
    try:
        return float(clean_str)
    except ValueError:
        return float('inf')  # Valor muy alto para comparaciones

def extract_product_data(soup):
    """
    Extrae información de los productos, incluyendo títulos, precios, links, descripción y materiales.

    Args:
        soup (BeautifulSoup): Objeto BeautifulSoup con el HTML parseado.

    Returns:
        pd.DataFrame: DataFrame con las columnas 'title', 'current_price', 'normal_price',
                      'description', 'materials', 'link'.
    """
    # Crear DataFrame vacío con las columnas deseadas
    df = pd.DataFrame(columns=['title', 'current_price', 'normal_price', 'description', 'materials', 'link'])

    # Extraer todas las tarjetas de producto con los nuevos selectores
    product_titles = soup.find_all('div', class_='product-card__title')
    product_price_containers = soup.find_all('div', class_='product-card__price')
    product_descriptions = soup.find_all('div', class_='product-card__subtitle')
    product_materials = soup.find_all('div', class_='product-card__messaging')
    product_links = soup.find_all('a', class_='product-card__link-overlay')

    # Determinar el número de productos (basado en títulos)
    num_items = len(product_titles)

    # Iterar sobre las tarjetas de productos y extraer información
    for i in range(num_items):
        # Pausa para evitar ser bloqueado por el servidor
        time.sleep(1)

        # Obtener el título del producto
        title = product_titles[i].text.strip() if i < len(product_titles) else "N/A"

        # Obtener los precios
        current_price = "N/A"
        discount_price = "N/A"
        
        if i < len(product_price_containers):
            price_wrapper = product_price_containers[i].find('div', class_='product-price__wrapper')
            
            if price_wrapper:
                # Buscar elementos de precio de diferentes clases
                price_elems = price_wrapper.find_all('div', class_=['es__styling', 'is--current-price', 'is--original-price'])
                
                # Filtrar y limpiar precios
                prices = [elem.text.strip() for elem in price_elems if elem and elem.text.strip()]
                
                # Lógica de asignación de precios
                if len(prices) >= 2:
                    # Ordenar precios de menor a mayor
                    sorted_prices = sorted(prices, key=clean_price)
                    current_price = sorted_prices[0]
                    normal_price = sorted_prices[-1]
                elif len(prices) == 1:
                    current_price = prices[0]
                    normal_price = current_price

        # Obtener la descripción del producto
        description = product_descriptions[i].text.strip() if i < len(product_descriptions) else "N/A"

        # Obtener los materiales
        materials = product_materials[i].text.strip() if i < len(product_materials) else "N/A"

        # Obtener el link del producto
        link = product_links[i]['href'] if i < len(product_links) else "N/A"

        # Agregar los datos al DataFrame
        df.loc[i] = [title, current_price, normal_price, description, materials, link]

    return df

In [3]:
df=extract_product_data(soup)

In [4]:
print(df)

                        title current_price normal_price  \
0          Nike Free Metcon 5      129,99 €     129,99 €   
1          Nike Free Metcon 6      129,99 €     129,99 €   
2               Nike Metcon 9      139,99 €     139,99 €   
3          Nike Free Metcon 6      129,99 €     129,99 €   
4               Nike Metcon 9      139,99 €     139,99 €   
5               Nike Metcon 9       97,99 €     139,99 €   
6               Nike Metcon 9      121,99 €     139,99 €   
7               Nike Metcon 9       97,99 €     139,99 €   
8               Nike Metcon 9      139,99 €     139,99 €   
9           Nike Metcon 9 AMP      149,99 €     149,99 €   
10       Nike Metcon 9 EasyOn      139,99 €     139,99 €   
11           Nike Metcon 1 OG      104,99 €     149,99 €   
12       Nike Metcon 9 EasyOn       97,99 €     139,99 €   
13          Nike Metcon 9 AMP      149,99 €     149,99 €   
14              Nike Metcon 9      139,99 €     139,99 €   
15         Nike Free Metcon 5      129,9

In [5]:
# Obtener la fecha y hora actual
fecha_hora = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

# Crear el nombre del archivo con la fecha y hora
nombre_archivo = f"productos_nike_{fecha_hora}.csv"

# Guardar el archivo CSV
df.to_csv(nombre_archivo, index=False, encoding='utf-8-sig')